In [1]:
import requests
from bs4 import BeautifulSoup
import datetime as dt
import pandas as pd

# 要爬的網址、api
spotify = 'https://spotifycharts.com/'
track_api = 'https://api.spotify.com/v1/tracks'
feature_api = 'https://api.spotify.com/v1/audio-features'
artists_api = 'https://api.spotify.com/v1/artists'


'''
設定要爬的國家
'''
r = requests.get('https://spotifycharts.com/regional/tw/weekly/latest')
soup = BeautifulSoup(r.text, 'html.parser')
countries = [(c['data-value'], c.text) for c in soup.find('div', {'class': 'chart-filters-list'}).find('div', {'data-type':'country'}).find('ul').findAll('li')]
countries_failed = []
print("取得所有國家")

for region, country in countries:
    
    try:
        time_start = dt.datetime.now()
        print('===================================================')
        print(f'{country} start!!')

        
        '''
        爬每週 top200 的名單
        '''
        fri = dt.date(2020, 11, 27)
        track_ids, w = [], 0

        r = requests.get(f'https://spotifycharts.com/regional/{region}/weekly/latest')
        soup = BeautifulSoup(r.text, 'html.parser')
        weeks = len(soup.find('div', {'class': 'chart-filters-list'}).find('div', {'data-type':'date'}).find('ul').findAll('li'))
        print(f'總共有{weeks}週')

        while w < weeks:  
            end, start = str(fri - dt.timedelta(days = 7*w)), str(fri - dt.timedelta(days = 7*(w+1)))
            week = f'{start}--{end}'
            w += 1  

            # 爬該週的top200名單
            url = f'{spotify}regional/{region}/weekly/{week}/download' 
            r = requests.get(url)
            table = r.text.split('\n')[2:-1]

            # 整理 top200 track_ids
            _ = [track.split(',')[-1].split('/')[-1] for track in table]
            track_ids += _

        track_ids = list(set(track_ids)) # 只取不重複的歌
        print(f'取得歌曲id 共有{len(track_ids)}首歌')

        
        '''
        爬 track_api 和 feature_api
        '''
        # 獲得 access_token (時間到會失效，每爬一個國家就重取一次token)
        CLIENT_ID = 'ad0d8699855a4e72b183657dc0f7d55a'
        CLIENT_SECRET = '3e66a60843e34360add3d4246c34fc52'
        AUTH_URL = 'https://accounts.spotify.com/api/token'
        auth_response = requests.post(AUTH_URL, {'grant_type': 'client_credentials', 'client_id': CLIENT_ID, 'client_secret': CLIENT_SECRET,})
        access_token = auth_response.json()['access_token']  # 取得token
        headers = {'Authorization': f'Bearer {access_token}'}

        data = pd.DataFrame(columns = ['track', 'feature'])
        track, feature = [], []
        i = 0 
        while i < len(track_ids):
            ids_t = ','.join(track_ids[i : i+50])  # tracks 一次最多50筆
            track += requests.get(track_api, params = {'ids':ids_t}, headers = headers).json()['tracks']

            if i%100 == 0:
                ids_f = ','.join(track_ids[i:i+100])  # audio-features 一次最多100筆
                feature += requests.get(feature_api, params = {'ids':ids_f}, headers = headers).json()['audio_features']

            i += 50

        data = pd.concat([data, pd.DataFrame(zip(track, feature), columns = ['track', 'feature'])], axis = 0)

        
        '''
        把爬蟲爬到的東西分到各個column裡
        '''
        # track_api 爬到的東西
        _ = [('track_name', 'name'), ('track_id', 'id'), ('popularity', 'popularity')]
        for col, fea in _:
            data[col] = data['track'].apply(lambda x: x[fea])
        _ = [('album_name', 'name'), ('album_id', 'id'), ('release_date', 'release_date')] 
        for col, fea in _:
            data[col] = data['track'].apply(lambda x: x['album'][fea])
        _ = [('artist_name', 'name'), ('artist_id', 'id')]    
        for col, fea in _:
            data[col] = data['track'].apply(lambda x: x['artists'][0][fea])

        # feature_api 爬到的東西    
        features = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature']
        for key in features:
            data[key] = data['feature'].apply(lambda x:x[key])

        print('取得track_api、feature_api資料')    

        
        '''
        從 artists_api 取得 genres 資訊
        '''    
        # 建立 artists_id、genres 對照表
        artist_genres = dict()
        artists_ids = list(data['artist_id'].unique())

        i = 0
        while i < len(artists_ids):
            ids = ','.join(artists_ids[i:i+50]) # artists 一次最多50筆

            r = requests.get(artists_api, params = {'ids':ids}, headers = headers)
            artist_genres.update({artist['id']:artist['genres'] for artist in r.json()['artists']})

            i += 50 

        # 取得 genres
        data['genres'] = data['artist_id'].map(artist_genres)
        print('取得genre資料')

        # 加上 country 資訊
        data['country'] = country
        
        
        '''
        輸出檔案
        '''
        # data[['track', 'feature']].to_csv(f'C:\\Users\\7334\\data_track_feature\\data_t_f_{country}', index = False)
        data = data.drop(['track', 'feature'], axis = 1)
        data.to_csv(f'C:\\Users\\7334\\data\\data_{country}.csv', index = False)
        
        print(dt.datetime.now() - time_start)
        print(f'{country} finish!!')
        
    except:
        countries_failed.append(country)
        print(dt.datetime.now() - time_start)
        print(f'{country} failed!!')
        continue

取得所有國家
Global start!!
總共有203週
取得歌曲id 共有3981首歌
0:03:30.975340
Global failed!!
United States start!!
總共有205週
取得歌曲id 共有4687首歌
取得track_api、feature_api資料
取得genre資料
0:03:20.879256
United States finish!!
United Kingdom start!!
總共有205週
取得歌曲id 共有4098首歌
取得track_api、feature_api資料
取得genre資料
0:04:01.922741
United Kingdom finish!!
Andorra start!!
總共有40週
取得歌曲id 共有151首歌
0:00:39.759045
Andorra failed!!
Argentina start!!
總共有205週
取得歌曲id 共有2448首歌
取得track_api、feature_api資料
取得genre資料
0:04:56.021313
Argentina finish!!
Austria start!!
總共有205週
取得歌曲id 共有4823首歌
取得track_api、feature_api資料
取得genre資料
0:05:51.818166
Austria finish!!
Australia start!!
總共有205週
取得歌曲id 共有3163首歌
取得track_api、feature_api資料
取得genre資料
0:05:45.131638
Australia finish!!
Belgium start!!
總共有205週
取得歌曲id 共有4328首歌
取得track_api、feature_api資料
取得genre資料
0:05:57.630223
Belgium finish!!
Bulgaria start!!
總共有205週
取得歌曲id 共有4299首歌
取得track_api、feature_api資料
取得genre資料
0:05:33.809622
Bulgaria finish!!
Bolivia start!!
總共有205週
取得歌曲id 共有2761首歌
0:05:11.746587
Bolivi

取得genre資料
0:04:30.756464
Paraguay finish!!
Romania start!!
總共有142週
取得歌曲id 共有2881首歌
取得track_api、feature_api資料
取得genre資料
0:03:12.653369
Romania finish!!
Russian Federation start!!
總共有20週
取得歌曲id 共有652首歌
取得track_api、feature_api資料
取得genre資料
0:00:29.740475
Russian Federation finish!!
Sweden start!!
總共有205週
取得歌曲id 共有4668首歌
取得track_api、feature_api資料
取得genre資料
0:03:54.831501
Sweden finish!!
Singapore start!!
總共有205週
取得歌曲id 共有2927首歌
取得track_api、feature_api資料
取得genre資料
0:04:00.242795
Singapore finish!!
Slovakia start!!
總共有205週
取得歌曲id 共有4291首歌
取得track_api、feature_api資料
取得genre資料
0:04:04.020084
Slovakia finish!!
El Salvador start!!
總共有205週
取得歌曲id 共有2250首歌
取得track_api、feature_api資料
取得genre資料
0:04:11.720105
El Salvador finish!!
Thailand start!!
總共有170週
取得歌曲id 共有2964首歌
取得track_api、feature_api資料
取得genre資料
0:03:51.363735
Thailand finish!!
Turkey start!!
總共有205週
取得歌曲id 共有2639首歌
取得track_api、feature_api資料
取得genre資料
0:04:00.158683
Turkey finish!!
Taiwan start!!
總共有205週
取得歌曲id 共有4271首歌
取得track_api、feature_ap

In [2]:
countries_failed

['Global',
 'Andorra',
 'Bolivia',
 'Ecuador',
 'Iceland',
 'Netherlands',
 'Panama',
 'Peru',
 'Philippines']